In [27]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
spark_version = "spark-3.2.0"
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [28]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Hashing").getOrCreate()

In [29]:
# Imports
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, StopWordsRemover

In [30]:
# Read in data from S3 Buckets

from pyspark import SparkFiles
df = spark.read.csv(SparkFiles.get("/content/IMDB Dataset.csv"),sep=",", escape='"', encoding="utf-8", quote='"',  header=True)

# Show DataFrame
df.show(10, truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [9]:
# Tokenize DataFrame
# tokened = Tokenizer(inputCol="review", outputCol="words")
# tokened_transformed = tokened.transform(df)
# tokened_transformed.show()

+--------------------+---------+--------------------+
|              review|sentiment|               words|
+--------------------+---------+--------------------+
|One of the other ...| positive|[one, of, the, ot...|
|A wonderful littl...| positive|[a, wonderful, li...|
|I thought this wa...| positive|[i, thought, this...|
|Basically there's...| negative|[basically, there...|
|Petter Mattei's "...| positive|[petter, mattei's...|
|Probably my all-t...| positive|[probably, my, al...|
|I sure would like...| positive|[i, sure, would, ...|
|This show was an ...| negative|[this, show, was,...|
|Encouraged by the...| negative|[encouraged, by, ...|
|If you like origi...| positive|[if, you, like, o...|
|Phil the Alien is...| negative|[phil, the, alien...|
|I saw this movie ...| negative|[i, saw, this, mo...|
|So im not a big f...| negative|[so, im, not, a, ...|
|The cast played S...| negative|[the, cast, playe...|
|This a fantastic ...| positive|[this, a, fantast...|
|Kind of drawn in ...| negat

In [10]:
# Remove stop words
# remover = StopWordsRemover(inputCol="words", outputCol="Wordsfiltered")
# removed_frame = remover.transform(tokened_transformed)
# removed_frame.show()

+--------------------+---------+--------------------+--------------------+
|              review|sentiment|               words|       Wordsfiltered|
+--------------------+---------+--------------------+--------------------+
|One of the other ...| positive|[one, of, the, ot...|[one, reviewers, ...|
|A wonderful littl...| positive|[a, wonderful, li...|[wonderful, littl...|
|I thought this wa...| positive|[i, thought, this...|[thought, wonderf...|
|Basically there's...| negative|[basically, there...|[basically, famil...|
|Petter Mattei's "...| positive|[petter, mattei's...|[petter, mattei's...|
|Probably my all-t...| positive|[probably, my, al...|[probably, all-ti...|
|I sure would like...| positive|[i, sure, would, ...|[sure, like, see,...|
|This show was an ...| negative|[this, show, was,...|[show, amazing,, ...|
|Encouraged by the...| negative|[encouraged, by, ...|[encouraged, posi...|
|If you like origi...| positive|[if, you, like, o...|[like, original, ...|
|Phil the Alien is...| ne

In [13]:
# # Run the hashing term frequency
# hashing = HashingTF(inputCol="Wordsfiltered", outputCol="hashedValues", numFeatures=pow(2,4))

# # Transform into a DF
# hashed_df = hashing.transform(removed_frame)
# hashed_df.show()

+--------------------+---------+--------------------+--------------------+--------------------+
|              review|sentiment|               words|       Wordsfiltered|        hashedValues|
+--------------------+---------+--------------------+--------------------+--------------------+
|One of the other ...| positive|[one, of, the, ot...|[one, reviewers, ...|(16,[0,1,2,3,4,5,...|
|A wonderful littl...| positive|[a, wonderful, li...|[wonderful, littl...|(16,[0,1,2,3,4,5,...|
|I thought this wa...| positive|[i, thought, this...|[thought, wonderf...|(16,[0,1,2,3,4,5,...|
|Basically there's...| negative|[basically, there...|[basically, famil...|(16,[0,1,2,3,4,5,...|
|Petter Mattei's "...| positive|[petter, mattei's...|[petter, mattei's...|(16,[0,1,2,3,4,5,...|
|Probably my all-t...| positive|[probably, my, al...|[probably, all-ti...|(16,[0,1,2,3,4,5,...|
|I sure would like...| positive|[i, sure, would, ...|[sure, like, see,...|(16,[0,1,2,3,4,5,...|
|This show was an ...| negative|[this, s

In [14]:
# # Fit the IDF on the data set 
# idf = IDF(inputCol="hashedValues", outputCol="features")
# idfModel = idf.fit(hashed_df)
# final_df = idfModel.transform(hashed_df)

In [15]:
# Show the TF-IDF features
# final_df.select("features").collect()

[Row(features=SparseVector(16, {0: 0.24, 1: 0.3852, 2: 0.2876, 3: 0.2831, 4: 0.3041, 5: 0.1196, 6: 0.1837, 7: 0.2665, 8: 0.2071, 9: 0.2364, 10: 0.4986, 11: 0.1333, 12: 0.1855, 13: 0.2165, 14: 0.306, 15: 0.0763})),
 Row(features=SparseVector(16, {0: 0.0873, 1: 0.3556, 2: 0.0885, 3: 0.0708, 4: 0.0702, 5: 0.1914, 6: 0.2067, 7: 0.0444, 8: 0.2071, 9: 0.1478, 10: 0.1385, 11: 0.1556, 12: 0.0927, 13: 0.1218, 14: 0.102, 15: 0.061})),
 Row(features=SparseVector(16, {0: 0.0873, 1: 0.1778, 2: 0.1106, 3: 0.1652, 4: 0.117, 5: 0.1675, 6: 0.1607, 7: 0.1554, 8: 0.1243, 9: 0.1182, 10: 0.1939, 11: 0.0667, 12: 0.0618, 13: 0.0947, 14: 0.153, 15: 0.0381})),
 Row(features=SparseVector(16, {0: 0.1309, 1: 0.0296, 2: 0.0885, 3: 0.118, 4: 0.0234, 5: 0.1196, 6: 0.1607, 8: 0.145, 9: 0.1478, 10: 0.2216, 11: 0.0667, 12: 0.0309, 13: 0.1218, 14: 0.051, 15: 0.0763})),
 Row(features=SparseVector(16, {0: 0.1309, 1: 0.2667, 2: 0.1106, 3: 0.0708, 4: 0.1871, 5: 0.2153, 6: 0.2755, 7: 0.1776, 8: 0.2071, 9: 0.1773, 10: 0.0277,

In [37]:
# Train test split
training, testing = df.randomSplit([0.7, 0.3])

In [38]:
# Imports
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, StringIndexer
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

In [39]:
# Create all the steps for the pipeline
label_indexer = StringIndexer(inputCol='review',outputCol='words')
tokenizer = Tokenizer(inputCol="words", outputCol="Wordsfiltered")
stopremove = StopWordsRemover(inputCol='Wordsfiltered',outputCol='hashedValues')
hashingTF = HashingTF(inputCol="hashedValues", outputCol='features')
lr = LogisticRegression(maxIter=20, regParam=0.001)

# Define pipeline
pipeline = Pipeline(stages=[label_indexer, tokenizer, stopremove, hashingTF, lr])

In [40]:
training.show()

+--------------------+---------+
|              review|sentiment|
+--------------------+---------+
|\b\b\b\bA Turkish...| positive|
|!!!! MILD SPOILER...| negative|
|!!!! POSSIBLE MIL...| negative|
|" Now in India's ...| positive|
|"... the beat is ...| positive|
|"2001: A Space Od...| positive|
|"200l: A Space Od...| positive|
|"9/11," hosted by...| positive|
|"A Mouse in the H...| positive|
|"A Slight Case of...| positive|
|"A Tale of Two Si...| positive|
|"A Thief in the N...| positive|
|"A bored televisi...| negative|
|"A death at a col...| negative|
|"A lot of the fil...| negative|
|"A trio of treasu...| negative|
|"A truly nice sto...| positive|
|"A wrong-doer is ...| positive|
|"A young man, rec...| negative|
|"A young woman un...| negative|
+--------------------+---------+
only showing top 20 rows



In [41]:
# Fit the pipeline to training reviews.
model = pipeline.fit(training)

IllegalArgumentException: ignored